In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from feature_extraction import Blob, Words
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from time import time
from pprint import pprint

In [46]:
data = pd.read_csv('processed_personalfinance.csv.bz2')
#data = pd.read_csv('processed_python.csv.bz2', encoding='ISO-8859-1', compression='bz2')
data.drop_duplicates('title', inplace=True)

data['gt5'] = data['ups'] > 5
data['gt20'] = data['ups'] > 20
data['gt10'] = data['ups'] > 10
data['gt50'] = data['ups'] > 50
data['gt100'] = data['ups'] > 100


data = data[data['subreddit'] == 'r/books']
train_X, test_X, train_y, test_y = train_test_split(data.title, 
                                                    data.gt10, 
                                                    test_size=0.20,
                                                    random_state=42)

print(data[data['gt10']==False].shape)
print(data[data['gt10']==True].shape)

(0, 11)
(0, 11)


In [36]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('words', Words()),
                      
            ('title', Pipeline([
                ('tfidf', TfidfVectorizer(ngram_range=(1,3), sublinear_tf=True, stop_words='english')),
                ('svd', TruncatedSVD(n_components=120)),
                ('normalize', MinMaxScaler(copy=False)),
                ('selector', SelectPercentile(f_classif, percentile=10))
            ])),
            
            ('blob', Pipeline([
                ('all', Blob()),
                ('minmax', MinMaxScaler()),
            ])),
            
            ])),
    ('clf', ExtraTreesClassifier()),
        ])

In [ ]:
# ExtraTreesClassifier(n_estimators=120, max_depth=5, max_features='sqrt', min_samples_leaf=5, min_samples_split=10)
# RandomForestClassifier(n_estimators=1200, max_depth=5, max_features='sqrt', min_samples_leaf=5, min_samples_split=10)

In [37]:
pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('words', Words()), ('title', Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...timators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [38]:
y = pipeline.predict(test_X)
accuracy_score(y_pred=y, y_true=test_y)

0.57845433255269318

In [31]:
cross_val_score(pipeline, train_X, train_y, cv=5)

array([ 0.78125   ,  0.78125   ,  0.78125   ,  0.78125   ,  0.80645161])

In [39]:
index = 2
print (test_X.iloc[index])
print (test_y.iloc[index])
print (y[index])
pipeline.predict(pd.Series(['Inspiration']))[0]

b'[help] unbound method save() must be called with MyModel instance as first argument'
False
False


False

In [9]:

'''
parameters = {
    'clf_n_estimators':(120,300,500,800,1200),
    'clf__max_depth':(5,8,15,25,30, None),
    'clf__min_samples_split':(2,5,10,15,100),
    'clf__min_samples_leaf':(1,2,5,10),
    'clf__max_features':('log2','sqrt',None),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(train_X, train_y)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


y = grid_search.predict(test_X)

accuracy_score(y_pred=y, y_true=test_y)
'''